# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for white-sounding names
sum(data[data.race=='w'].call)

235.0

In [7]:
# number of callbacks for white-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [6]:
data.shape

(4870, 65)

**1. What test is appropriate for this problem? Does CLT apply?**

T-test is appropriate as the population means for the 2 groups are not known. The test should be a two sample test as we're trying to find the difference in mean between 2 distinct groups.

CLT applies as the sample size is large and the observations are independent.

**2. What are the null and alternate hypotheses?**

The null hypothesis is that the difference in means between callback rates for black and white sounding names is zero.
The alternate hypothesis is that there is a statistically significant difference in means between the 2 groups.

In [9]:
def draw_bs_reps(data, func, size=1):
    bs_reps = np.empty(size)
    for i in range(size):
        bs_reps[i] = func(np.random.choice(data, size=len(data)))
    return bs_reps

**3. Compute margin of error, confidence interval, and p-value.**

In [19]:
w_call = data[data.race=='w'].call
b_call = data[data.race=='b'].call

In [20]:
print(len(data) == len(w_call)+len(b_call))

True


In [21]:
w_mean = np.mean(w_call)
b_mean = np.mean(b_call)
mean_diff = w_mean - b_mean
print(w_mean)
print(b_mean)
print(mean_diff)

0.09650924056768417
0.0644763857126236
0.03203285485506058


In [31]:
# p value
from scipy.stats import ttest_ind
t, p = ttest_ind(w_call, b_call)

print('t score:',t)
print('p value =',p)

t score: 4.114705290861751
p value = 3.940802103128886e-05


The t score is high (>1.96 for 95% confidence interval) and the p value is very low (when compared to .05 for 95% confidence interval). **The null hypothesis can thus be rejected. This suggests that there is a statistically significant difference between call back rates between the 2 races.**

In [32]:
# margin of error 
print('Margin of error for white sounding names: ', stats.sem(w_call))
print('Margin of error for black sounding names: ', stats.sem(b_call))

Margin of error for white sounding names:  0.005985301397503016
Margin of error for black sounding names:  0.0049781434352911685


In [34]:
# margin of error calculated in a different way (using the formula directly)
margin_error_w = np.std(w_call)/np.sqrt(len(w_call))
print('Margin of error for white sounding names: ', margin_error_w)
margin_error_b = np.std(b_call)/np.sqrt(len(b_call))
print('Margin of error for black sounding names: ',margin_error_b)

Margin of error for white sounding names:  0.0059840016981803105
Margin of error for black sounding names:  0.004977108869798699


In [35]:
# 95% confidence interval
from scipy.stats import t
w_conf_int = t.interval(0.95, df=len(w_call)-1, loc=w_mean, scale=stats.sem(w_call))
b_conf_int = t.interval(0.95, df=len(b_call)-1, loc=b_mean, scale=stats.sem(b_call))
print('95% conf interval for mean for white sounding names: ', w_conf_int)
print('95% conf interval for mean for black sounding names: ', b_conf_int)

95% conf interval for mean for white sounding names:  (0.08477242903143205, 0.1082460521039363)
95% conf interval for mean for black sounding names:  (0.054714549604450616, 0.07423822182079658)


**4. Write a story describing the statistical significance in the context or the original problem.**

From the above statistics, with the t score being more than 4 standard deviations and the p-value being far less than 0.05, **we can reject the null hypothesis and reach the conclusion that the alternate hypothesis is true**. There is a statistically significant difference in the call back rates for white sounding vs black sounding names.

Moreover, the confidence intervals for the 2 groups don't even overlap. This strengthens the above conclusion.

**5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?**

This analysis does not mean that race/name is the most important factor. We haven't even considered any of the other columns - education, number of years of experience, field of expertise etc. 

If that were the objective, we would want to explore the data more to analyze the impact of the other columns on the callback rate. We would want to check for any existing patterns for candidates with successful callbacks.